<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projeto Proposto 
</h1>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este projeto baseia-se no Estudo das Técnicas de Machine Learning - Regras de Associação. </i>       
</p>  

> CONSULTAR: 

https://www.youtube.com/watch?v=YGEYty0xYc0&list=PL-t7zzWJWPtzhZtI-bWWHFtCfxtjmBdIW&index=26&ab_channel=UniversoDiscreto

https://www.youtube.com/watch?v=Mq5HPAFXrOI&list=PL-t7zzWJWPtzhZtI-bWWHFtCfxtjmBdIW&index=27&ab_channel=UniversoDiscreto

# <font color='red' style='font-size: 40px;'>Library and Data </font>

In [1]:
import pandas as pd 
import numpy as np
import scipy as sp 
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time
import datetime
import plotly.express as px 
import plotly.graph_objects as go
import warnings



%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore') # Remove Warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='red' style='font-size: 40px;'>1.0 O que são Regras de Associação? </font>

> CONSULTAR:

https://daniel-s-amador.medium.com/regras-de-associa%C3%A7%C3%A3o-b02bb9ce6ea3

https://www.youtube.com/watch?v=E6q_oNkAjAo&ab_channel=ronaldoframos


## 1.1) O que são Regras de Associação?

<img src = 'img460.png' width = 70%>
<img src = 'img461.png' width = 70%>
<img src = 'img462.png' width = 70%>
<img src = 'img463.png' width = 70%>
<img src = 'img464.png' width = 70%>


# <font color='red' style='font-size: 40px;'>2.0 Algoritmo APRIORI </font>

## 2.1) O que é o APRIORI?

> CONSULTAR:

https://medium.com/@bernardo.costa/uma-introdu%C3%A7%C3%A3o-ao-algoritmo-apriori-60b11293aa5a#:~:text=O%20Apriori%20trabalha%20com%20o,um%20subconjunto%20de%20Itens%20frequentes.

https://www.youtube.com/watch?v=YGEYty0xYc0&t=18s&ab_channel=UniversoDiscreto

### 1) Conceitos Fudamentais

> 1. Primeiro, definem-se a frequência de cada produtos em relação a base crua.

> 2. Posteriormente, itera-se a aplicação da Métrica de Suporte e, através de um Threshold, descobre-se os produtos mais frequentes e aqueles os quais definiremos Regras de Associação.

> 3. Criação de Hipóteses sobre Associação, ou seja, Combinações de Todos para Todos

* T -> C, A? | T, A -> C?
* A -> T, C? | T, C -> A?
* C -> T, A? | A, C -> T?

> 4. Agora, calcula-se a Métrica de Confiança dos Produtos selecionados 

> 5. Após o calculo da Métrica de Confiança, aplica-se um Threshold, sobrando apenas as Regras Associativas fortes e que passaram no Threshold de Confiança

> 6. Por fim, calcula-se a Métrica de Lift, pois ela é uma Métrica que não dá importância apenas ao item antecessor, mas sim a presença dos itens no geral. 

* Se Lift(X=>Y) > 1, então o conjunto Y é provável de ser comprado quando X for comprado.

* Se Lift(X=>Y) ≤ 1, então NÃO é provável que Y seja comprado, caso X seja comprado.

In [2]:
df = pd.read_csv('Groceries_dataset.csv')
print(df.shape)
print(df.dtypes)
df.head()

(38765, 3)
Member_number       int64
Date               object
itemDescription    object
dtype: object


,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


* Valores Missing 

In [3]:
df.isnull().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

* Date para Datetime 

In [4]:
df['Date'] = pd.to_datetime(df['Date'])
print(df.dtypes)

Member_number               int64
Date               datetime64[ns]
itemDescription            object
dtype: object


* Frequência de Itens

In [5]:
df.itemDescription.value_counts().head(10)

whole milk          2502
other vegetables    1898
rolls/buns          1716
soda                1514
yogurt              1334
root vegetables     1071
tropical fruit      1032
bottled water        933
sausage              924
citrus fruit         812
Name: itemDescription, dtype: int64

* Reshape da Base 

In [6]:
df.head()

,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-05-01,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-01-02,whole milk


In [7]:
df_reshape = df.copy()
df_reshape = df_reshape.groupby(['Member_number', 'itemDescription'])['itemDescription'].count().unstack().fillna(0).applymap(lambda x: 1 if x> 0 else 0)
df_reshape

itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,...,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
Member_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1001,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
1002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0
4998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
from mlxtend.frequent_patterns import apriori, association_rules

In [15]:
# Gerando associação

associacao = apriori(df_reshape, min_support=0.05, use_colnames=True)

# Colocando em ordem dos mais frequentes

associacao.sort_values(by = 'support', ascending = False).head(10)

c:\Users\leova\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
55,0.46,(whole milk)
37,0.38,(other vegetables)
41,0.35,(rolls/buns)
47,0.31,(soda)
56,0.28,(yogurt)
51,0.23,(tropical fruit)
42,0.23,(root vegetables)
5,0.21,(bottled water)
44,0.21,(sausage)
116,0.19,"(other vegetables, whole milk)"


In [16]:
# Criando regras de associação

regras = association_rules(associacao, metric = 'lift')

# Colocando em ordem de suporte, cofnianca ou grau de associação (lift)

resultado = regras.sort_values(by = 'lift', ascending = False)
resultado.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
197,(bottled water),"(other vegetables, whole milk)",0.21,0.19,0.06,0.26,1.37,0.02,1.10
196,"(other vegetables, whole milk)",(bottled water),0.19,0.21,0.06,0.29,1.37,0.02,1.11
198,(other vegetables),"(bottled water, whole milk)",0.38,0.11,0.06,0.15,1.33,0.01,1.04
195,"(bottled water, whole milk)",(other vegetables),0.11,0.38,0.06,0.50,1.33,0.01,1.25
241,(yogurt),"(other vegetables, whole milk)",0.28,0.19,0.07,0.25,1.33,0.02,1.08
236,"(other vegetables, whole milk)",(yogurt),0.19,0.28,0.07,0.38,1.33,0.02,1.15
199,(whole milk),"(bottled water, other vegetables)",0.46,0.09,0.06,0.12,1.31,0.01,1.03
194,"(bottled water, other vegetables)",(whole milk),0.09,0.46,0.06,0.60,1.31,0.01,1.35
250,"(whole milk, rolls/buns)",(yogurt),0.18,0.28,0.07,0.37,1.30,0.02,1.14
251,(yogurt),"(whole milk, rolls/buns)",0.28,0.18,0.07,0.23,1.30,0.02,1.07
